In [14]:
words = open("words-en.txt", "r")
words_list = []
for word in words.readlines(): 
    words_list.append(word)

# Almacenamiento sin compresión

Veamos qué longitud máxima de palabra encontramos:

In [15]:
max_word_length = 0
max_word = ""
for word in words_list:
  if len(word) > max_word_length:
    max_word = word
    max_word_length = len(word)

In [16]:
print(max_word_length)
print(max_word)

32
dichlorodiphenyltrichloroethane



Si almacenáramos el vocabulario sin comprimir, necesitaríamos utilizar 32 bytes para cada término, lo que multiplicado por la cantidad de palabras de la lista, nos da el espacio total requerido.

In [17]:
total_space = len(words_list) * 32
print(f"Total space without compression: {total_space} bytes = {total_space/1024/1024} MB")

Total space without compression: 11357984 bytes = 10.831817626953125 MB


# Compresión con Dictionary-as-String

In [32]:
total_characters = 0
word_count = 0
for word in words_list:  
  total_characters += len(word)
  word_count += 1

print(f"Total characters: {total_characters}")
print(f"Space required: {total_characters/1024/1024} MB")

Total characters: 3712426
Space required: 3.540445327758789 MB
354937


Sumando la cantidad de caracteres que tienen los términos de la colección y considerando que cada caracter requiere 1 byte para ser almacenado, requeriríamos 3712426 bytes - 3.54 MB - de espacio para almacenarlo como dictionary-as-string.

A su vez, debemos tener en cuenta que para saber dónde comienza y termina cada término, requerimos información adicional como un puntero al comienzo de cada uno. Con esto, dedicando 4 bytes a cada puntero, 1 por término, tenemos en total:

In [34]:
pointers_size = len(words_list) * 4
print(f"Pointers size: {pointers_size} = {pointers_size/1024/1024} MB")
total_daas = total_characters + pointers_size
print(f"Total dictionary-as-string size: {total_daas} = {total_daas/1024/1024} MB")

Pointers size: 1419748 = 1.3539772033691406 MB
Total dictionary-as-string size: 5132174 = 4.89442253112793 MB


Por lo tanto, el tamaño total requerido para el almacenamiento es de 4,89 MB. Esto representa menos de la mitad utilizada por una estrategia sin compresión.

# Límite de tamaño de palabra
Analizando las palabras, hemos visto que la mayor tiene una longitud de 32 caracteres. Contemos cuántas ocurrencias de cada longitud de palabra hay en el archivo.

In [19]:
word_lengths = []
for word in words_list:
  word_len = len(word)
  word_lengths.append(word_len)

In [ ]:
%pip install matplotlib
from matplotlib import pyplot as plt
# Creating histogram
fig, axs = plt.subplots(1, 1,
                        figsize =(7, 5),
                        tight_layout = True)
 
axs.hist(word_lengths)
 
# Show plot
plt.show()

In [ ]:
# importing library
%pip install pandas
import pandas as pd

# converting list to array
series = pd.Series(word_lengths)

In [25]:
series.describe()

count    354937.000000
mean         10.459394
std           2.922625
min           1.000000
25%           8.000000
50%          10.000000
75%          12.000000
max          32.000000
dtype: float64

Utilizando estadística descriptiva básica, podemos ver que la mayor parte de los términos de la colección se encuentran rondando los 10 caracteres, y que si bien son pocos los que tienen menos de 3, casi ninguno supera los 20 caracteres, salvo unas pocas excepciones.

Con esto en consideración, uno podría definir una longitud máxima de palabra en 20 o 21, sabiendo que son pocas las que quedarán fuera del vocabulario. Específicamente, la siguiente cantidad:

In [26]:
greater_than_20 = series[series > 20]

In [27]:
len(greater_than_20)

653

Considerando la cantidad de palabras que tenía el archivo, este valor representa el siguiente porcentaje:

In [28]:
len(greater_than_20) / len(words_list) * 100

0.18397631128904565

Es decir, solo quedarían fuera del vocabulario el 0.18% de los términos. 


Veamos cómo descendería el tamaño del vocabulario sin comprimir si tomamos un largo de 20 para cada término:

In [29]:
less_or_equal_20 = series[series <= 20]
new_size = len(less_or_equal_20)*20
print(f"Vocabulary size (term length = 20): {new_size} bytes = {new_size / 1024 / 1024} MB")
print(f"Reduction: {(total_space - new_size)/ total_space * 100}%")

Vocabulary size (term length = 20): 7085680 bytes = 6.7574310302734375 MB
Reduction: 37.61498519455566%


Vemos que la reducción es bastante significativa, dado que ganamos 12 bytes por cada término del vocabulario. Por lo tanto, sin utilizar dictionary-as-string resulta conveniente limitar el tamaño de cada término.

Verifiquemos qué ocurre si quitamos estos términos pero almacenándolos con dictionary-as-string:

In [31]:
size = 0
for word in words_list:
  word_len = len(word)
  if word_len <= 20:
    size += word_len
print(f"New vocabulary size: {size/1024/1024} MB")
print(f"Old vocabulary size: {total_characters/1024/1024} MB")
print(f"Reduction: {(total_characters - size)/ total_characters * 100}%")

New vocabulary size: 3.526836395263672 MB
Old vocabulary size: 3.540445327758789 MB
Reduction: 0.3843847661879321%


Podemos observar que la reducción es muy baja, por lo que probablemente nos perjudicaríamos más quitando estos términos que manteniéndolos, siendo así capaces de responder por ellos en las queries de los usuarios.